### 지도학습

#### 분류 - 이진분류
- 목표변수가 2개 항목인 것
- 목표변수 : Survived

In [1]:
import pandas as pd

In [2]:
df_TFD = pd.read_csv('../../../datasets/TitanicFromDisaster_train.csv')
df_TFD[:2]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


#### 전처리

In [3]:
df_TFD.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
df_TFD_extract = df_TFD[['Survived', 'Pclass', 'Age']]
df_TFD_extract.isnull().sum()

Survived      0
Pclass        0
Age         177
dtype: int64

In [5]:
# age null값 존재 -> regression으로 null값을 채울 수 있음. 여기선 drop함
df_TFD_extract_preprocess = df_TFD_extract.dropna()
df_TFD_extract_preprocess[:2]

,Survived,Pclass,Age
0,0,3,22.0
1,1,1,38.0


#### Scaling & Encoding

##### Encoding with OneHotEncoding
- 범주형 데이터가 존재할 경우 이를 수치화(벡터화)하는 대표적인 방법

In [6]:
# 범주형 'Pclass' 확인
df_TFD_extract_preprocess['Pclass'].value_counts()

3    355
1    186
2    173
Name: Pclass, dtype: int64

In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
oneHotEncoder = OneHotEncoder()   #인스턴스화
oneHotEncoder.fit(df_TFD_extract_preprocess[['Pclass']])   #'Pclass'를 기준으로 교육시킴, 해당 항목을 학습한 것

OneHotEncoder()

In [9]:
columns_name = oneHotEncoder.categories_   # value_counts()로 학인했던 범주형(1-3등급)의 항목을 1,2,3으로 부여함
columns_name

[array([1, 2, 3], dtype=int64)]

In [10]:
encoded_data = oneHotEncoder.transform(df_TFD_extract_preprocess[['Pclass']]).toarray()
encoded_data.shape

(714, 3)

In [11]:
# columns_name, encoded_data를 사용해 dataframe을 만들기
# 병합을 위해서 numpy array를 dataframe으로 바꿔줌
df_encoded_data = pd.DataFrame(data=encoded_data, columns=oneHotEncoder.get_feature_names_out(['Pclass']))
df_encoded_data[:2]

,Pclass_1,Pclass_2,Pclass_3
0,0.0,0.0,1.0
1,1.0,0.0,0.0


In [12]:
df_TFD_extract_preprocess[['Pclass']][:3]

# 위의 결과와 동일하게 나옴
# [0., 0., 1.], 3등급이므로 자신이 해당하는 세번째만 1로 표시됨.
# [1., 0., 0.], 1등급이므로 자신이 해당하는 첫번째만 1로 표시됨.
# [0., 0., 1.], 3등급이므로 자신이 해당하는 세번째만 1로 표시됨.

,Pclass
0,3
1,1
2,3


In [13]:
df_encoded_data.index, df_encoded_data.shape

(RangeIndex(start=0, stop=714, step=1), (714, 3))

In [14]:
df_TFD_extract_preprocess.isnull().sum()

Survived    0
Pclass      0
Age         0
dtype: int64

In [15]:
# df_encoded_data = pd.get_dummies(df_TFD_extract_preprocess['Pclass'], prefix='Pclass')
## prefix='Pclass' -> 모든 column 이름 앞에 고정적으로 붙음

In [16]:
# 원래 데이터에 다른 데이터 합치기
# axis=1 : 행을 기준으로 옆으로 붙음
df_TFD_extract_preprocess = pd.concat([df_TFD_extract_preprocess.reset_index(drop=True), df_encoded_data.reset_index(drop=True)], axis=1)
df_TFD_extract_preprocess[:2]

,Survived,Pclass,Age,Pclass_1,Pclass_2,Pclass_3
0,0,3,22.0,0.0,0.0,1.0
1,1,1,38.0,1.0,0.0,0.0


In [17]:
df_TFD_extract_preprocess.shape

(714, 6)

##### Scaling

In [18]:
df_TFD_extract_preprocess.columns

Index(['Survived', 'Pclass', 'Age', 'Pclass_1', 'Pclass_2', 'Pclass_3'], dtype='object')

In [19]:
target = df_TFD_extract_preprocess['Survived']

In [20]:
features = df_TFD_extract_preprocess.drop(columns=['Survived', 'Pclass'])

In [21]:
features.columns

Index(['Age', 'Pclass_1', 'Pclass_2', 'Pclass_3'], dtype='object')

##### -MinMaxScaler(정규화)
- 모든 값을 0~1 사이의 값으로 바꾸는 것

In [22]:
from sklearn.preprocessing import MinMaxScaler

In [23]:
minMaxScaler = MinMaxScaler()   # 인스턴스화
features = minMaxScaler.fit_transform(features)
features.shape   # 714개의 row에 4개의 값이 들어감

(714, 4)

#### 정형화 단계
- 데이터를 머신에 넣기 전 목표변수와 설명변수를 분리
- 데이터 분할(Split)
    * 머신러닝 모델을 훈련하고 성능을 측정할 때 훈련 데이터와 테스트 데이터 비율을 8:2로 설정한다.
    * train_test_split() : 8:2 비율로 만들어주는 fuction
    * 데이터가 500개 미만일 경우 split을 하는 것보다 데이터를 더 모으는 게 좋다.

In [24]:
# split 하지 않은 정형화
# target_train = df_TFD_extract_preprocess['Survived']
# features_train = df_TFD_extract_preprocess[['Pclass', 'Age']]   # label(=feature)
# target_train.shape, features_train.shape

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=111)
features_train.shape, target_train.shape, features_test.shape, target_test.shape

((535, 4), (535,), (179, 4), (179,))

#### 모델학습

In [27]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()  # 인스턴스화
model.fit(features_train, target_train)  # 모델 훈련은 fit() fuction을 이용함 -> fit(feature, target)

LogisticRegression()

In [28]:
model.coef_, model.intercept_
# 정규화 이전값 : (array([[-1.257039902, -0.04521969]]), array([3.78621796]))

(array([[-2.59255724,  1.11141847,  0.02997489, -1.14146766]]),
 array([0.86447724]))

#### 예측

In [29]:
# 값을 넣어서 확인해보기
df_TFD_extract_preprocess[10:15]   # index가 features_train과 같다.

,Survived,Pclass,Age,Pclass_1,Pclass_2,Pclass_3
10,1,1,58.0,1.0,0.0,0.0
11,0,3,20.0,0.0,0.0,1.0
12,0,3,39.0,0.0,0.0,1.0
13,0,3,14.0,0.0,0.0,1.0
14,1,2,55.0,0.0,1.0,0.0


In [30]:
model.predict(features_train[10:15])
# 위에서는 0이 3개 1이 2개
# 결과: array([0, 0, 0, 0, 0]) 60% 맞음

# scaling 한 후 결과: array([0, 0, 0, 1, 0])

array([0, 0, 0, 1, 0], dtype=int64)

In [31]:
# predict_proba() : 확률을 알 수 있음
model.predict_proba(features_train[10:15])

# [0.52507531, 0.47492469] -> 앞은 0에 대한 열, 뒤는 1에 대한 열
# 0에 대한 확률이 더 높기 때문에 위에서 0으로 결과가 나온 것

array([[0.59715375, 0.40284625],
       [0.63564456, 0.36435544],
       [0.75219168, 0.24780832],
       [0.47500723, 0.52499277],
       [0.73984819, 0.26015181]])

#### 평가

In [32]:
target_train_predict = model.predict(features_train)
target_train_predict.shape  # target_train.shape과 동일

(535,)

In [33]:
from sklearn.metrics import accuracy_score

In [34]:
# 정확도 평가 (실제값, 예측값을 넣으면 정확도가 평가된다)
accuracy_score(target_train, target_train_predict)  # 교내 시험
# 정규화 이전 : 0.7065420560747664

0.708411214953271

In [35]:
target_test_predict = model.predict(features_test)
target_test_predict.shape   # target_test.shape과 동일

(179,)

In [36]:
accuracy_score(target_test, target_test_predict)   # 교외 시험

# 교내 시험, 교외 시험의 두 개의 차이가 많이 나면 모델 성능에 문제가 있다고 판단
# 0.7, 0.65는 0.05정도의 차이이므로 양호함
# 정규화 이전 : 0.65921787709

0.6480446927374302

##### **성능지표
1. accuracy(정확도) : 정확하게 분류한 데이터 수 / 전체 데이터 수
    * 실제 데이터 분포는 unbalance하기 때문에 accuracy만으로 평가하는 것은 무의미함.
    * 예측값과 살제값이 얼마나 동일한지에 대한 비율로 결정
2. precision(정밀도)
    * 모델이 True라고 분류한 것 중 실제 True인 비율
3. recall(재현율)
    * 실제 True인 것 중에서 모델이 True라고 예측한 것의 비율
    * 정밀도와 재현율은 상호보완적, 두 지표가 모두 높을수록 좋은 지표
4. F1 Score

In [37]:
from sklearn.metrics import classification_report

In [38]:
print(classification_report(target_train, target_train_predict))   # vs code의 파이썬에서 사용되는 코드라서 print로 감싸줘야 모양이 이상하지 않게 나옴

              precision    recall  f1-score   support

           0       0.72      0.82      0.77       312
           1       0.69      0.55      0.61       223

    accuracy                           0.71       535
   macro avg       0.70      0.69      0.69       535
weighted avg       0.71      0.71      0.70       535



In [39]:
print(classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           0       0.72      0.71      0.71       112
           1       0.53      0.55      0.54        67

    accuracy                           0.65       179
   macro avg       0.63      0.63      0.63       179
weighted avg       0.65      0.65      0.65       179



#### 오차 행렬(confusion matrix)

In [40]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [41]:
confusion_matrix(target_train, target_train_predict)

# 정규화 이전
# TN(True Negative) : 347 (예측을 negative로 했는데 실제도 negative)
# FP(False Positive) : 77 (예측을 positive로 했는데 실제는 negative)
# FN(False Negative) : 140 (예측은 negative로 했는데 실제는 positive)
# TP(True Positive) : 150 (예측은 positive로 했는데 실제도 positvie)

array([[257,  55],
       [101, 122]], dtype=int64)

In [42]:
precision_score(target_train, target_train_predict)

0.6892655367231638

In [43]:
recall_score(target_train, target_train_predict)

0.547085201793722

In [44]:
f1_score(target_train, target_train_predict)

0.6100000000000001

#### 서비스

In [45]:
df_TFD_extract[100:103]

,Survived,Pclass,Age
100,0,3,28.0
101,0,3,NaN
102,0,1,21.0


In [46]:
# 예측 대상자 입력값 -> Pclass:3, Age:28.0
# oneHotEncoder : 인스턴스화 되어 있음, fit()을 통해 Pclass 학습되어 있는 상태.
encoder_pclass_ = oneHotEncoder.transform([[3]]).toarray() # 교육시킬 때 이차원 형태로 넣어줬으므로 이차원 형태[[]]로 넣어줌
encoder_pclass_, encoder_pclass_.flatten()

C:\Users\koreavc\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


(array([[0., 0., 1.]]), array([0., 0., 1.]))

In [47]:
# [[28.0, [0., 0., 1.]]]
import numpy as np
inputs_data = np.concatenate(([28.0],encoder_pclass_.flatten()))
inputs_data

array([28.,  0.,  0.,  1.])

In [48]:
model.predict([inputs_data]) # 목표변수 얻기

array([0], dtype=int64)